<a href="https://colab.research.google.com/github/vascotreis/Cat_identifier/blob/main/catbreed_classification_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##*Installing the necessary libraries for this project*##

In [ ]:
## Data set was taken from https://www.robots.ox.ac.uk/~vgg/data/pets/

In [ ]:
#!pip install imageio
#!pip install scikit-image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##*connecting the notebook with your google drive*##
###*This is done so that I can access the data set that is stored in google drive*###

In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


##*Declaring the path of the files in the drive*##
##*Also the path where the model will be saved*##

In [ ]:
path="/content/drive/MyDrive/Test_the_Keras/Projecto/CATS 2/"
model_dir='/content/drive/MyDrive/Test_the_Keras/Projecto/'

##*Importing all the necessary libraries and modules for the program*##

In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.layers import Dense, Flatten, Dropout
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

import matplotlib.pyplot as plt
import tensorflow as tf
from imageio import imread
from skimage.transform import resize

from sklearn.preprocessing import LabelEncoder

##*Creating an empty list to store all the name of breeds*##

In [ ]:
# create a list to store the breeds
breeds = []

##*The naming convention of the dataset is as follows*##
####BreedName_number.jpg###
###the below code splits the name into two portions### 
####*1. before the   "_"*####
####*2. After the    "_"*#### 
###all unique strings before "_" are appended to the list of breeds###

In [ ]:
# iterate through the images in the folder
for image in os.listdir(path):
    if image.endswith('.jpg') or image.endswith('.jpeg') or image.endswith('.png'):
        # extract the breed name from the image name
        breed = image.split("_")[0]
        # add the breed to the list if it is not already in the list
        if breed not in breeds:
            breeds.append(breed)


print(breeds)

['Bengal', 'Sphynx', 'Bombay', 'Egyptian', 'Persian', 'Ragdoll', 'Siamese', 'Maine', 'Birman', 'Abyssinian', 'Russian', 'British']


###you can see that there are 12 breeds in the given daaset###

###*creating a dictionary to map breed to an integer for computation purposes*###

> Bloco com avanço



In [ ]:
# create a dictionary to map breed names to integers
breed_mapping = {breed: i for i, breed in enumerate(breeds)}

In [ ]:
print(breed_mapping)

{'Bengal': 0, 'Sphynx': 1, 'Bombay': 2, 'Egyptian': 3, 'Persian': 4, 'Ragdoll': 5, 'Siamese': 6, 'Maine': 7, 'Birman': 8, 'Abyssinian': 9, 'Russian': 10, 'British': 11}


###*Two empty lists are created for storing the image data and the respective labels*###

In [ ]:
# create the data and labels arrays
data = []
labels = []

###*Images in the dataset may be of varying size and resolution, to create a model for prediction using Neural Networks all the images should be of a standard size*###
####*- resolution of 224 x 224 was chosen for this project, because all research I did images I saw it seams a universal size and also this works well with the VGG16 model I plan to test for this project*#### 
####*- all the images should be resized to this resolution before storing into data list*####


In [ ]:
# iterate through the images in the folder
for image in os.listdir(path):
    if image.endswith('.jpg') or image.endswith('.jpeg') or image.endswith('.png'):
        # load the image
        img = cv2.imread(path+image)
        # resize the image
        try:
          # resizing the image
          img = cv2.resize(img, (224, 224))
          # add the image to the data list
          data.append(img)
          # extract the breed name from the image name
          breed = image.split("_")[0]
          # add the corresponding label to the labels list
          labels.append(breed_mapping[breed])
        except Exception as e:
          print(f"could not resize {image}")
       

        

could not resize Egyptian_Mau_191.jpg
could not resize Egyptian_Mau_177.jpg
could not resize Egyptian_Mau_139.jpg
could not resize Egyptian_Mau_145.jpg
could not resize Abyssinian_34.jpg
could not resize Egyptian_Mau_167.jpg


In [ ]:
# Delete all of this images so it wont scruew my code

###*please note that not all images could be resized, some of the images which are shown above were not resizable hence we had to ignore those*### 

In [ ]:
print(f"total nnumber of images available : {len(data)}")

total nnumber of images available : 2402


###*In order to train the model we require the data and labels as numpy arrays*###
 ###*Numpy arrays are more compact than python lists, which uses less memory. Numpy is also not just more efficient but convienient. There are a lot of vector and matrix operations in Numpy. There are also things built into Numpy such as FFT's, convolutions, statistics, histograms, etc*###
####*- hence converting the python lists : data and labels in to numpy arrays*#### 

In [ ]:
data = np.array(data)
labels = np.array(labels)
print(f"Data : {data.shape} , lables : {labels.shape}")

Data : (2402, 224, 224, 3) , lables : (2402,)


###*In order to make sure that the training set as well as the test set have representatives of all Breeds the data was split into numerous sample sizes randomly*###

####*Train and test set are later gathered from this randomized pool of data*####

# I saw this while researcho Keras 
# Stratified ShuffleSplit cross-validator. Provides train/test indices to split data in train/test sets. This cross-validation object is a merge of StratifiedKFold and ShuffleSplit, which returns stratified randomized folds. The folds are made by preserving the percentage of samples for each class.#. 

###*In order to make sure that the training set as well as the test set have representatives of all Breeds the data was split into numerous sample sizes randomly*###

####*Train and test set are later gathered from this randomized pool of data*####

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

# Your data and labels stored in X and y respectively
X = data
y = labels

# Define the number of splits you want
n_splits = 15

# Create an instance of the StratifiedShuffleSplit class
sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=0.15, random_state=42)

# Use the split method to get the train and test indices
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

print(f"training set: {len(X_train)} , test set : {len(X_test)}")

training set: 2041 , test set : 361


In [ ]:
# VGG16 model, Will researching I found this model and seemed easy to apply so I used some snipets from this web site https://towardsdatascience.com/step-by-step-vgg16-implementation-in-keras-for-beginners-a833c686ae6c and ChatGBT to compose my code

In [ ]:
# 

###*download the VGG16 model and define the input data shape*###

In [ ]:
# Create the base model of VGG16 model
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

###*- Converting data to tensors is necessary as tensors are the basic data structure used in TensorFlow and are required for building and training a machine learning model.*###

###*- Normalizing the data by dividing it by 255.0 helps in scaling the values of the data between 0 and 1, which can help the model learn better.*###

###*- One-hot encoding the labels transforms the labels into a binary matrix representation where each column corresponds to a single category. This encoding is needed for multiclass classification problems as it allows the model to predict the class of a sample by providing the probability of each class.*###

In [ ]:
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)

X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)


X_train = X_train / 255.0
X_test = X_test / 255.0

# One-hot encode the labels
y_train = to_categorical(y_train, len(breeds))
y_test = to_categorical(y_test, len(breeds))


###*- This code block freezes the layers of the base VGG16 model, meaning that the weights of these layers will not be updated during training. This helps to prevent overfitting by retaining the pre-trained feature extraction capabilities of the base model.*###

In [ ]:
# Freeze the layers of the base model
for layer in vgg16_model.layers:
    layer.trainable = False

###*- The code below creates a Sequential model in Keras.*###
###*- The VGG16 model is then added on top of the base model by calling model.add(vgg16_model).*###
###*- The Flatten layer is then added, which is used to flatten the multi-dimensional input from the previous layer into a one-dimensional array to be processed by the dense layers.*###
###*- The first Dense layer is added with 256 units and ReLU activation. The number of units determines the dimensionality of the output space and ReLU activation is used for the activation function.*###
###*- The Dropout layer is added with a rate of 0.5, which is used to prevent overfitting by randomly dropping some neurons during training.*###
###*- The final Dense layer is added with the number of units equal to the number of different breeds and with a Softmax activation function. The Softmax activation function is used to ensure that the sum of all outputs is 1, so that the outputs can be interpreted as probabilities.*###
###* -The model is then compiled by specifying the optimizer (Adam), loss function (categorical crossentropy), and evaluation metrics (accuracy).*###
###* -A ModelCheckpoint is created to save the best model after each epoch during training.*###
###* -Finally, the model is fit on the training data for 25 epochs with a batch size of 128, with the validation data and the ModelCheckpoint being passed to the fit method as arguments.*###

## The softmax function, also known as softargmax[1]: 184  or normalized exponential function,[2]: 198  converts a vector of K real numbers into a probability distribution of K possible outcomes. It is a generalization of the logistic function to multiple dimensions, and used in multinomial logistic regression. The softmax function is often used as the last activation function of a neural network to normalize the output of a network to a probability distribution over predicted output classes, based on Luce's cho

In [ ]:
# Create a Sequential model
model = Sequential()

# Add the VGG16 model on top of the base model
model.add(vgg16_model)

model.add(Flatten())
# Add a Dense layer with 128 units and ReLU activation
model.add(Dense(256, activation='relu'))
# Add a Dropout layer with a rate of 0.5
model.add(Dropout(0.5))
# Add a Dense layer with the number of units equal to the number of breeds and Softmax activation
model.add(Dense(len(breeds), activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Create a checkpoint to save the best model
checkpoint = ModelCheckpoint(model_dir+'best_model.h5', save_best_only=True)

# Fit the model on the training data
model.fit(X_train, y_train, epochs=25, batch_size=128, validation_data=(X_test, y_test), callbacks=[checkpoint])


Epoch 1/25
16/16 [==============================] - 43s 2s/step - loss: 3.7221 - accuracy: 0.1627 - val_loss: 2.1072 - val_accuracy: 0.3380
Epoch 2/25
16/16 [==============================] - 12s 732ms/step - loss: 1.9568 - accuracy: 0.3258 - val_loss: 1.7902 - val_accuracy: 0.4792
Epoch 3/25
16/16 [==============================] - 12s 735ms/step - loss: 1.6473 - accuracy: 0.4297 - val_loss: 1.5293 - val_accuracy: 0.5346
Epoch 4/25
16/16 [==============================] - 12s 737ms/step - loss: 1.3988 - accuracy: 0.5272 - val_loss: 1.3262 - val_accuracy: 0.6011
Epoch 5/25
16/16 [==============================] - 12s 736ms/step - loss: 1.2034 - accuracy: 0.5948 - val_loss: 1.2531 - val_accuracy: 0.6482
Epoch 6/25
16/16 [==============================] - 12s 742ms/step - loss: 1.0827 - accuracy: 0.6306 - val_loss: 1.1367 - val_accuracy: 0.6537
Epoch 7/25
16/16 [==============================] - 12s 748ms/step - loss: 0.9204 - accuracy: 0.6997 - val_loss: 1.0670 - val_accuracy: 0.6759
Ep

###*Loading the model to use in predicting some images*###

In [ ]:
from keras.models import load_model

In [ ]:
#Load the best model
model = load_model('/content/drive/MyDrive/Test_the_Keras/Projecto/best_model.h5')


###*loading the images for whihc I want to get the breed*###

In [ ]:
#Load the image you want to classify
# Birman
img9 = cv2.imread('/content/drive/MyDrive/Test_the_Keras/Projecto/CATS 2/Birman_79.jpg')
# Bengal
img2 = cv2.imread('/content/drive/MyDrive/Test_the_Keras/Projecto/CATS 2/Bengal_60.jpg')
#Bombay 
img3 = cv2.imread('/content/drive/MyDrive/Test_the_Keras/Projecto/CATS 2/Bombay_183.jpg')
#British
img4 = cv2.imread('/content/drive/MyDrive/Test_the_Keras/Projecto/CATS 2/British_Shorthair_45.jpg')
#Leia
img5 = cv2.imread('/content/drive/MyDrive/Test_the_Keras/Projecto/British_Shortair_150.jpg')


###*Defined a function for finding the breed*###

In [ ]:
def predict_breed(model, image):
  
  #Convert the image to a numpy array
  img = cv2.resize(image, (224, 224))
  
  #Expand the dimensions of the image to (1, 224, 224, 3)
  img = np.expand_dims(img, axis=0)
  
  result_prob = model.predict(img)
  result = result_prob.argmax(axis=-1)

  predicted_class_index = np.argmax(result_prob)

  #Get the label of the predicted breed
  predicted_breed = breeds[predicted_class_index]



  print(f"The predicted breed is : {predicted_breed}\n")



In [ ]:
# should predict Abyssinian
predict_breed(model,img9)
# should predict Bengal
predict_breed(model,img2)
# Should predict Bombay
predict_breed(model,img3)
# Should predict British_shortair
predict_breed(model,img4)
#
predict_breed(model,img5)

1/1 [==============================] - 0s 18ms/step
The predicted breed is : Birman

1/1 [==============================] - 0s 18ms/step
The predicted breed is : Bengal

1/1 [==============================] - 0s 18ms/step
The predicted breed is : Bombay

1/1 [==============================] - 0s 16ms/step
The predicted breed is : Bengal

1/1 [==============================] - 0s 19ms/step
The predicted breed is : Birman



###*In conclusion, atleast for the cases tried so far the model seems to be predicting the correct breed*###
###*However please note that the model accuracy is approximately 70% hance it is bound to make some mistakes in general*###

In [ ]:
### As you can see it identify a British short air as a Bengal
